<a href="https://colab.research.google.com/github/cadenlpicard/car_review_sentiment_analysis/blob/main/Car_Review_Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional
from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight
from sklearn.metrics import classification_report, accuracy_score
from tensorflow.keras.utils import to_categorical

# Load training data
train_data = pd.read_excel('/content/drive/MyDrive/Machine Learning/Competitions/car_review_sentiment/input/Train_data.xlsx')  # Replace with the path to your training data .xlsx file

# Preprocess training data
train_data = train_data.dropna()  # Remove rows with missing values
texts = train_data['Review'].astype(str).values  # Extract reviews
labels = train_data['Target'].values  # Extract labels (0: Negative, 1: Neutral, 2: Positive)

# Convert labels to categorical (one-hot encoding)
labels = to_categorical(labels, num_classes=3)

# Tokenize the text
max_words = 5000  # Maximum number of words to keep in the vocabulary
max_len = 100  # Maximum length of a review after padding
tokenizer = Tokenizer(num_words=max_words, oov_token="<OOV>")
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)
padded_sequences = pad_sequences(sequences, maxlen=max_len, padding='post', truncating='post')

# Split data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(padded_sequences, labels, test_size=0.2, random_state=42)

# Calculate class weights to handle imbalance
class_weights = class_weight.compute_class_weight('balanced', classes=np.unique(train_data['Target']), y=train_data['Target'])
class_weights_dict = dict(enumerate(class_weights))

# Build a more complex model with Bidirectional LSTM
model = Sequential([
    Embedding(input_dim=max_words, output_dim=64, input_length=max_len),
    Bidirectional(LSTM(64, return_sequences=True)),
    Dropout(0.5),
    LSTM(32),
    Dense(32, activation='relu'),
    Dropout(0.5),
    Dense(3, activation='softmax')  # Softmax for multi-class classification
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model with class weights
epochs = 10  # Increase epochs for better training
batch_size = 32
history = model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_val, y_val), class_weight=class_weights_dict)

# Evaluate model on validation set
y_val_pred_prob = model.predict(X_val)
y_val_pred = np.argmax(y_val_pred_prob, axis=1)  # Convert probabilities to class predictions
y_val_true = np.argmax(y_val, axis=1)  # Convert one-hot encoded labels to class numbers
print("Validation Accuracy:", accuracy_score(y_val_true, y_val_pred))
print("\nClassification Report on Validation Set:\n", classification_report(y_val_true, y_val_pred, target_names=["Negative", "Neutral", "Positive"]))

# Load the separate test data (only has "Review" column)
test_data = pd.read_excel('/content/drive/MyDrive/Machine Learning/Competitions/car_review_sentiment/input/Test_features.xlsx')  # Replace with the path to your test data .xlsx file
test_data = test_data.dropna()  # Remove rows with missing values
test_texts = test_data['Review'].astype(str).values  # Extract reviews

# Preprocess the test data
test_sequences = tokenizer.texts_to_sequences(test_texts)
padded_test_sequences = pad_sequences(test_sequences, maxlen=max_len, padding='post', truncating='post')

# Predict on the test set
y_pred_prob = model.predict(padded_test_sequences)
y_pred = np.argmax(y_pred_prob, axis=1)  # Convert probabilities to class predictions

# Save predictions to CSV with review and predicted target
predictions_save_path = '/content/drive/MyDrive/Machine Learning/Competitions/car_review_sentiment/predictions.csv'  # Replace with desired save path
predictions_df = pd.DataFrame({'Review': test_texts, 'Predicted Label': y_pred})
predictions_df.to_csv(predictions_save_path, index=False)

print(f"Predictions saved to {predictions_save_path}")


In [ ]:
from google.colab import drive
drive.mount('/content/drive')